In [1]:
import json
import os
from library_dicom.dicom_processor.model.Series import Series 
from library_dicom.dicom_processor.model.SeriesPT import SeriesPT
from library_dicom.dicom_processor.model.SeriesCT import SeriesCT
from library_dicom.dicom_processor.model.csv_reader.MaskBuilder import MaskBuilder

import numpy as np
import SimpleITK as sitk

In [2]:
json_path = '/media/salim/DD 2To/AHL2011_NIFTI/AHL2011_list_path_csv.json'

In [3]:
data = []
with open(json_path) as json_file : 
    reader = json.load(json_file)
    for info in reader :
        for path in reader[info] :
            data.append(path)

In [4]:
dataset = []
for i in range(0,len(data),5):
    subliste = []
    subliste.append(data[i])
    subliste.append(data[i+1])
    subliste.append(data[i+2])
    subliste.append(data[i+3])
    subliste.append(data[i+4])
    dataset.append(subliste)

In [5]:
print(dataset[0])


['/media/salim/DD 2To/AHL2011_Validated_DICOMS/13011101331015/1.2.124.113532.10.149.172.5.20131107.93455.7725981/1.2.840.113619.2.290.3.2500492204.268.1383804922.866.3/1.2.840.113619.2.290.3.2500492204.268.1383804922.866.3', 'CT', '/media/salim/DD 2To/AHL2011_Validated_DICOMS/13011101331015/1.2.124.113532.10.149.172.5.20131107.93455.7725981/1.2.840.113619.2.290.3.2500492204.268.1383804923.167/1.2.840.113619.2.290.3.2500492204.268.1383804923.167', 'PT', 'con_da_nov 7_2013.csv']


In [ ]:
#genere la liste des series qui ont le unconstant spacing 
liste_unconstant_z_spacing = []
for liste in dataset : 
    for i in range(0,4,2) : 
        serie_objet = Series(liste[i])
        serie_objet.get_numpy_array()
        if serie_objet.get_z_spacing() == 'Unconstant Spacing' : 
            liste_unconstant_z_spacing.append(liste)

#save it as json

In [6]:
#read json with unconstant spacing liste
path = '/media/salim/DD 2To/AHL2011_NIFTI/AHL2011_spacing_problem.json'
liste_unconstant_spacing = []
with open(path) as json_file: 
    reader = json.load(json_file)
    for info in reader : 
        liste_unconstant_spacing.append(info)



In [11]:
#calcul écart max des z spacing d'une serie 
def ecart_spacing(liste):
    ecart = []
    ecart.append(abs(liste[0] - liste[1]))
    
    for j in range(2, len(liste)):
        ecart.append(abs(liste[j-1] - liste[j]))
    return np.max(ecart)

In [14]:
#print la liste des ecart max des z spacing de toutes les séries unconstant spacing 
ecart_list = []
for liste in liste_unconstant_spacing : 
    for i in range(0,4,2) : 
        serie_objet = Series(liste[i])
        serie_objet.get_numpy_array()
        if serie_objet.get_z_spacing() == 'Unconstant Spacing' : 
            space = serie_objet.calculate_z_spacing()
            print(space)
            max_ecart = ecart_spacing(space)
            print("ecart max : ", max_ecart)
            ecart_list.append(max_ecart)
            print(ecart_list)

, 0.06000000000000005, 0.009999999999999787, 0.009999999999999787, 0.009999999999999787, 0.009999999999999787, 0.009999999999999787, 2.0, 0.009999999999999787, 2.0, 0.009999999999999787, 0.40000000000000036, 0.009999999999999787, 0.40000000000000036, 0.009999999999999787, 0.009999999999999787, 0.009999999999999787, 0.009999999999999787, 0.009999999999999787, 0.009999999999999787, 0.08000000000000007, 0.08000000000000007, 0.08000000000000007, 0.08000000000000007, 0.08000000000000007, 0.08000000000000007, 0.08000000000000007, 0.08000000000000007, 0.08000000000000007, 0.08000000000000007, 0.08000000000000007, 0.08000000000000007, 0.08000000000000007, 0.08000000000000007, 0.009999999999999787, 0.009999999999999787, 0.009999999999999787, 0.009999999999999787, 0.08000000000000007, 0.06000000000000005, 0.06000000000000005, 0.06000000000000005, 0.06000000000000005, 0.06000000000000005, 1.0, 0.009999999999999787, 0.009999999999999787, 0.009999999999999787, 0.009999999999999787, 0.00999999999999

In [6]:

nifti_directory = '/media/salim/DD 2To/AHL2011_NIFTI'


csv_directory = '/media/salim/DD 2To/AHL2011_CSV_DATA/AHL2011-CSV'


for serie in dataset : 
    print(serie)

    for i in range(1,4,2): #inde 1 et 3 => CT OU PT


        if serie[i] == 'CT' : 
            serie_ct_objet = SeriesCT(serie[i - 1])
            study_uid = serie_ct_objet.get_series_details()['study']['StudyInstanceUID']
            serie_ct_objet.get_numpy_array()
            if serie_ct_objet.get_z_spacing() != 'Unconstant Spacing' : 
                filename = study_uid+'_'+'nifti_'+'CT'+'.nii'
                serie_ct_objet.export_nifti(os.path.join(nifti_directory, filename))
                

            else : 
                print("Unconstant Spacing à la série CT " , dataset.index(serie))



    

        if serie[i] == 'PT' :
            print('serie PT, generation du mask avec fichier csv associée')
            serie_pt_objet = SeriesPT(serie[i - 1])
            study_uid = serie_pt_objet.get_series_details()['study']['StudyInstanceUID']

            nifti_array = serie_pt_objet.get_numpy_array()
            if serie_pt_objet.get_z_spacing() != 'Unconstant Spacing' : 
                filename = study_uid+'_'+'nifti_'+'PT'+'.nii'
                serie_pt_objet.export_nifti(os.path.join(nifti_directory, filename))

                size_matrix = serie_pt_objet.get_size_matrix()
                print(size_matrix)

                mask_objet = MaskBuilder(os.path.join(csv_directory, serie[4]), size_matrix)
                mask_4D = mask_objet.mask_array
                #si le masque est ok, alors generer nifti :
                #filename = study_uid+'_'+'nifti'+'mask_4D'+'.nii'
                #serie_pt_objet.export_nifti(os.path.join(nifti_directory, filename), mask_4D)
                print(mask_objet.calcul_suv(serie_pt_objet.get_numpy_array()))
                if mask_objet.is_correct_suv(serie_pt_objet.get_numpy_array()) == False : 
                    print("erreur calcul suv, philips")
                    print(mask_objet.ecart_suv_max(nifti_array))

            else : print("Unconstant Spacing à la série PT " , dataset.index(serie))



['/media/salim/DD 2To/AHL2011_Validated_DICOMS/13011101331015/1.2.124.113532.10.149.172.5.20131107.93455.7725981/1.2.840.113619.2.290.3.2500492204.268.1383804922.866.3/1.2.840.113619.2.290.3.2500492204.268.1383804922.866.3', 'CT', '/media/salim/DD 2To/AHL2011_Validated_DICOMS/13011101331015/1.2.124.113532.10.149.172.5.20131107.93455.7725981/1.2.840.113619.2.290.3.2500492204.268.1383804923.167/1.2.840.113619.2.290.3.2500492204.268.1383804923.167', 'PT', 'con_da_nov 7_2013.csv']
serie PT, generation du mask avec fichier csv associée
[256, 256, 299]
{1: {'SUV_max': 4.52, 'SUV_mean': 2.66, 'SD': 0.55}, 2: {'SUV_max': 8.19, 'SUV_mean': 4.58, 'SD': 1.14}, 3: {'SUV_max': 3.58, 'SUV_mean': 2.16, 'SD': 0.53}, 4: {'SUV_max': 8.34, 'SUV_mean': 5.24, 'SD': 1.28}, 5: {'SUV_max': 9.27, 'SUV_mean': 5.31, 'SD': 1.14}, 6: {'SUV_max': 5.78, 'SUV_mean': 3.27, 'SD': 0.63}, 7: {'SUV_max': 11.71, 'SUV_mean': 6.21, 'SD': 1.23}, 8: {'SUV_max': 13.11, 'SUV_mean': 7.16, 'SD': 1.38}, 9: {'SUV_max': 12.21, 'SUV_m

KeyboardInterrupt: 

In [8]:
list_manufacturer = []
for serie in dataset : 
#serie CT / PT 
    for i in range(2):
        #print(i)
        #recupérer study IUD et CT PT dans le tri des données, dans le json 
        serie_object = Series(serie[i])
        serie_data = serie_object.get_series_details()
        modality = serie_data['series']['Modality'] #CT ou PT

        if modality == 'PT' :
            serie_pt = SeriesPT(serie[i])
            list_manufacturer.append(serie_pt.get_series_details()['series']['Manufacturer'])

print(list_manufacturer)

['GE MEDICAL SYSTEMS', 'Philips Medical Systems', 'Philips Medical Systems', 'Philips Medical Systems', 'Philips Medical Systems', 'Philips Medical Systems', 'Philips Medical Systems', 'Philips Medical Systems', 'Philips Medical Systems', 'Philips Medical Systems', 'GE MEDICAL SYSTEMS', 'SIEMENS', 'SIEMENS', 'SIEMENS', 'SIEMENS', 'SIEMENS', 'SIEMENS', 'SIEMENS', 'SIEMENS', 'SIEMENS', 'SIEMENS', 'SIEMENS', 'SIEMENS', 'SIEMENS', 'SIEMENS', 'SIEMENS', 'SIEMENS', 'SIEMENS', 'SIEMENS', 'SIEMENS', 'SIEMENS', 'SIEMENS', 'SIEMENS', 'GE MEDICAL SYSTEMS', 'GE MEDICAL SYSTEMS', 'GE MEDICAL SYSTEMS', 'GE MEDICAL SYSTEMS', 'GE MEDICAL SYSTEMS', 'GE MEDICAL SYSTEMS', 'CPS', 'CPS', 'CPS', 'CPS', 'CPS', 'CPS', 'CPS', 'CPS', 'CPS', 'CPS', 'CPS', 'CPS', 'CPS', 'Philips Medical Systems', 'Philips Medical Systems', 'Philips Medical Systems', 'Philips Medical Systems', 'Philips Medical Systems', 'Philips Medical Systems', 'Philips Medical Systems', 'Philips Medical Systems', 'Philips Medical Systems', 'Phi

In [16]:
len([74.0, 1.25, 0.009999999999999787, 0.009999999999999787, 0.009999999999999787, 0.009999999999999787, 0.009999999999999787, 0.009999999999999787, 0.009999999999999787, 0.009999999999999787, 0.009999999999999787, 0.009999999999999787, 0.009999999999999787, 0.06000000000000005, 0.06000000000000005, 0.06000000000000005, 0.06000000000000005, 0.06000000000000005, 0.06000000000000005, 0.3999999999999999, 0.009999999999999787, 0.009999999999999787, 1.25, 1.25, 1.25, 1.25, 2.5, 0.009999999999999787, 0.009999999999999787, 0.009999999999999787, 0.009999999999999787, 0.009999999999999787, 0.009999999999999787, 0.009999999999999787, 0.009999999999999787, 0.009999999999999787, 0.7999999999999998, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.25, 0.06000000000000005, 0.06000000000000005, 0.08000000000000007, 0.06000000000000005, 0.06000000000000005, 0.06000000000000005, 0.06000000000000005, 0.009999999999999787, 0.009999999999999787, 0.009999999999999787, 0.009999999999999787, 0.009999999999999787, 2.0, 0.009999999999999787, 2.0, 0.009999999999999787, 0.40000000000000036, 0.009999999999999787, 0.40000000000000036, 0.009999999999999787, 0.009999999999999787, 0.009999999999999787, 0.009999999999999787, 0.009999999999999787, 0.009999999999999787, 0.08000000000000007, 0.08000000000000007, 0.08000000000000007, 0.08000000000000007, 0.08000000000000007, 0.08000000000000007, 0.08000000000000007, 0.08000000000000007, 0.08000000000000007, 0.08000000000000007, 0.08000000000000007, 0.08000000000000007, 0.08000000000000007, 0.08000000000000007, 0.009999999999999787, 0.009999999999999787, 0.009999999999999787, 0.009999999999999787, 0.08000000000000007, 0.06000000000000005, 0.06000000000000005, 0.06000000000000005, 0.06000000000000005, 0.06000000000000005, 1.0, 0.009999999999999787, 0.009999999999999787, 0.009999999999999787, 0.009999999999999787, 0.009999999999999787, 0.009999999999999787, 0.009999999999999787, 0.009999999999999787, 0.009999999999999787, 0.009999999999999787, 0.009999999999999787])

105